In [ ]:
# !pip install pandas xlrd langdetect -q

In [ ]:
import pandas as pd
import langdetect

In [ ]:
DATA_NAMES = ['ensisijainen', 'toissijainen']

DATA = DATA_NAMES[0]

print(DATA)

In [ ]:
df = pd.read_excel(f'data/raw/{DATA}_anonymized.xlsx', index_col=0)
df.head()

In [ ]:
# drop columns where all values are NaN
df = df.dropna(axis=1, how='all')

In [ ]:
pd.set_option('display.max_rows', 20000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

In [ ]:
df.head(500)

In [ ]:
def notnan(x):
    return x == x

In [ ]:
organisaatio1, organisaatio2, organisaatio3, question = None, None, None, None

row_dicts = []

for org1, org2, org3, q, _, answer in df[3:].itertuples(name=None, index=False):
    
    # rivit joissa organisaatiotaso on määritelty

    if notnan(org1):
        if org1.strip().endswith('Total'):
            continue

        organisaatio1 = org1
        organisaatio2 = org2
        organisaatio3 = org3
        question = q
        
    elif notnan(org2):
        if org2.strip().endswith('Total'):
            continue

        organisaatio2 = org2
        organisaatio3 = org3
        question = q

        
    elif notnan(org3):
        if org3.strip().endswith('Total'):
            continue

        organisaatio3 = org3
        question = q
        
    
    # rivit joissa kysymys vaihtuu
    elif notnan(q):
        if q.strip().endswith('Total'):
            continue

        question = q
        
    # muuten rivi on vastaus kysymykseen
    elif notnan(answer):
        row_dict = {'organisaatio1': organisaatio1,
                    'organisaatio2': organisaatio2,
                    'organisaatio3': organisaatio3, 
                    'question': question, 
                    'answer': answer}

        row_dicts.append(row_dict)

row_df = pd.DataFrame(row_dicts)

In [ ]:
row_df.head(10)

In [ ]:
row_df['question'].value_counts()

In [ ]:
def get_lang(row):
    try:
        lang = langdetect.detect(row['answer'])
        
        # Most of the text is finnish or swedish but langdetect does not work very well for short texts.
        # Let's assume that everything that is not Swedish is Finnish
        if lang != 'sv':
            lang = 'fi'

    except langdetect.lang_detect_exception.LangDetectException:
        lang = 'fi'

    return lang

In [ ]:
row_df['lang'] = row_df.apply(get_lang, axis=1)

In [ ]:
row_df.head()

In [ ]:
row_df['lang'].value_counts()

In [ ]:
question = {
    'ensisijainen': 'Mitä työssäni tarvittavaa osaamista kehitän?',
    'toissijainen': 'Mitä osaamista kehitän sen lisäksi?'
}[DATA]

In [ ]:
q = (row_df['question'] == question)
fi = (row_df['lang'] == 'fi')

row_df_fi = row_df.loc[q & fi]

row_df_fi.to_csv(f'./data/processed/{DATA}.csv')

In [ ]:
row_df_fi.head(10)